In [1]:
from google.colab import drive
drive.mount('/gdrive', force_remount=True)

Mounted at /gdrive


In [2]:
dir_ = '/gdrive/My Drive/SUBMISSION MODEL/'

In [3]:
raw_data_dir = dir_+'2. data/'
processed_data_dir = dir_+'2. data/processed/'
log_dir = dir_+'4. logs/'
model_dir = dir_+'5. models/'

In [4]:
ver, KKK = 'priv', 0

In [5]:
STORES = ['CA_1', 'CA_2', 'CA_3', 'CA_4', 'TX_1', 'TX_2', 'TX_3', 'WI_1', 'WI_2', 'WI_3']
CATS = ['HOBBIES','HOUSEHOLD', 'FOODS']

In [6]:
# General imports
import numpy as np
import pandas as pd
import os, sys, gc, time, warnings, pickle, psutil, random

# custom imports
from multiprocessing import Pool

warnings.filterwarnings('ignore')

In [7]:
def seed_everything(seed=0):
    random.seed(seed)
    np.random.seed(seed)

    
## Multiprocess Runs
def df_parallelize_run(func, t_split):
    num_cores = np.min([N_CORES,len(t_split)])
    pool = Pool(num_cores)
    df = pd.concat(pool.map(func, t_split), axis=1)
    pool.close()
    pool.join()
    return df

In [8]:
# Read data
def get_data_by_store(store, dept):
    
    df = pd.concat([pd.read_pickle(BASE),
                    pd.read_pickle(PRICE).iloc[:,2:],
                    pd.read_pickle(CALENDAR).iloc[:,2:]],
                    axis=1)
    
    df = df[df['d']>=START_TRAIN]
    

    df = df[(df['store_id']==store) & (df['cat_id']==dept)]

    df2 = pd.read_pickle(MEAN_ENC)[mean_features]
    df2 = df2[df2.index.isin(df.index)]
        
    df3 = pd.read_pickle(LAGS).iloc[:,3:]
    df3 = df3[df3.index.isin(df.index)]
    
    df = pd.concat([df, df2], axis=1)
    del df2
    
    df = pd.concat([df, df3], axis=1)
    del df3
    
    features = [col for col in list(df) if col not in remove_features]
    df = df[['id','d',TARGET]+features]
    
    df = df.reset_index(drop=True)
    
    return df, features

# Recombine Test set after training
def get_base_test():
    base_test = pd.DataFrame()

    for store_id in STORES:
        for state_id in CATS:
            temp_df = pd.read_pickle(processed_data_dir+'test_'+store_id+'_'+state_id+'.pkl')
            temp_df['store_id'] = store_id
            temp_df['cat_id'] = state_id
            base_test = pd.concat([base_test, temp_df]).reset_index(drop=True)
    
    return base_test

def make_lag(LAG_DAY):
    lag_df = base_test[['id','d',TARGET]]
    col_name = 'sales_lag_'+str(LAG_DAY)
    lag_df[col_name] = lag_df.groupby(['id'])[TARGET].transform(lambda x: x.shift(LAG_DAY)).astype(np.float16)
    return lag_df[[col_name]]


def make_lag_roll(LAG_DAY):
    shift_day = LAG_DAY[0]
    roll_wind = LAG_DAY[1]
    lag_df = base_test[['id','d',TARGET]]
    col_name = 'rolling_mean_tmp_'+str(shift_day)+'_'+str(roll_wind)
    lag_df[col_name] = lag_df.groupby(['id'])[TARGET].transform(lambda x: x.shift(shift_day).rolling(roll_wind).mean())
    return lag_df[[col_name]]

In [9]:
import lightgbm as lgb
lgb_params = {
                    'boosting_type': 'gbdt',
                    'objective': 'tweedie',
                    'tweedie_variance_power': 1.1,
                    'metric': 'rmse',
                    'subsample': 0.5,
                    'subsample_freq': 1,
                    'learning_rate': 0.015,
                    'num_leaves': 2**8-1,
                    'min_data_in_leaf': 2**8-1,
                    'feature_fraction': 0.5,
                    'max_bin': 100,
                    'n_estimators': 3000,
                    'boost_from_average': False,
                    'verbose': -1
                }

In [10]:
VER = 1                        
SEED = 42                      
seed_everything(SEED)
lgb_params['seed'] = SEED
N_CORES = psutil.cpu_count()


#LIMITS and const
TARGET      = 'sales'
START_TRAIN = 700 
END_TRAIN   = 1941 - 28*KKK
P_HORIZON   = 28        
USE_AUX     = False

remove_features = ['id','cat_id', 'state_id','store_id',
                   'date','wm_yr_wk','d',TARGET]
mean_features   = ['enc_store_id_dept_id_mean','enc_store_id_dept_id_std',
                   'enc_item_id_store_id_mean','enc_item_id_store_id_std'] 

ORIGINAL = raw_data_dir
BASE     = processed_data_dir+'grid_part_1.pkl'
PRICE    = processed_data_dir+'grid_part_2.pkl'
CALENDAR = processed_data_dir+'grid_part_3.pkl'
LAGS     = processed_data_dir+'lags_df_28.pkl'
MEAN_ENC = processed_data_dir+'mean_encoding_df.pkl'


SHIFT_DAY  = 28
N_LAGS     = 15
LAGS_SPLIT = [col for col in range(SHIFT_DAY,SHIFT_DAY+N_LAGS)]
ROLS_SPLIT = []
for i in [1,7,14]:
    for j in [7,14,30,60]:
        ROLS_SPLIT.append([i,j])

In [11]:
for store_id in STORES:
    for state_id in CATS:
        print('Train', store_id, state_id)

        grid_df, features_columns = get_data_by_store(store_id, state_id)

        train_mask = grid_df['d']<=END_TRAIN
        valid_mask = train_mask&(grid_df['d']>(END_TRAIN-P_HORIZON))
        preds_mask = (grid_df['d']>(END_TRAIN-100)) & (grid_df['d'] <= END_TRAIN+P_HORIZON)

        train_data = lgb.Dataset(grid_df[train_mask][features_columns], 
                           label=grid_df[train_mask][TARGET])

        valid_data = lgb.Dataset(grid_df[valid_mask][features_columns], 
                           label=grid_df[valid_mask][TARGET])

        grid_df = grid_df[preds_mask].reset_index(drop=True)
        keep_cols = [col for col in list(grid_df) if '_tmp_' not in col]
        grid_df = grid_df[keep_cols]

        d_sales = grid_df[['d','sales']]
        substitute = d_sales['sales'].values
        substitute[(d_sales['d'] > END_TRAIN)] = np.nan
        grid_df['sales'] = substitute

        grid_df.to_pickle(processed_data_dir+'test_'+store_id+'_'+state_id+'.pkl')
        del grid_df, d_sales, substitute

        seed_everything(SEED)
        estimator = lgb.train(lgb_params,
                              train_data,
                              valid_sets = [valid_data],
                              verbose_eval = 100
                              
                              )
        
        display(pd.DataFrame({'name':estimator.feature_name(),
                              'imp':estimator.feature_importance()}).sort_values('imp',ascending=False).head(25))
        
        model_name = model_dir+'lgb_model_'+store_id+'_'+state_id+'_v'+str(VER)+'.bin'
        pickle.dump(estimator, open(model_name, 'wb'))

        
        del train_data, valid_data, estimator
        gc.collect()

        MODEL_FEATURES = features_columns

Train CA_1 HOBBIES
[100]	valid_0's rmse: 2.16008
[200]	valid_0's rmse: 2.083
[300]	valid_0's rmse: 2.04832
[400]	valid_0's rmse: 2.0186
[500]	valid_0's rmse: 1.99018
[600]	valid_0's rmse: 1.96107
[700]	valid_0's rmse: 1.93312
[800]	valid_0's rmse: 1.90586
[900]	valid_0's rmse: 1.8788
[1000]	valid_0's rmse: 1.85102
[1100]	valid_0's rmse: 1.82548
[1200]	valid_0's rmse: 1.79899
[1300]	valid_0's rmse: 1.77329
[1400]	valid_0's rmse: 1.74885
[1500]	valid_0's rmse: 1.72413
[1600]	valid_0's rmse: 1.69874
[1700]	valid_0's rmse: 1.6768
[1800]	valid_0's rmse: 1.65103
[1900]	valid_0's rmse: 1.62562
[2000]	valid_0's rmse: 1.60377
[2100]	valid_0's rmse: 1.58243
[2200]	valid_0's rmse: 1.56054
[2300]	valid_0's rmse: 1.54238
[2400]	valid_0's rmse: 1.5228
[2500]	valid_0's rmse: 1.50205
[2600]	valid_0's rmse: 1.48298
[2700]	valid_0's rmse: 1.46435
[2800]	valid_0's rmse: 1.44363
[2900]	valid_0's rmse: 1.4273
[3000]	valid_0's rmse: 1.41012


,name,imp
0,item_id,109778
22,tm_w,24050
21,tm_d,23691
14,event_name_1,19353
55,rolling_mean_180,19107
56,rolling_std_180,19049
57,rolling_mean_tmp_1_7,18201
54,rolling_std_60,17641
48,rolling_std_7,17626
50,rolling_std_14,16832


Train CA_1 HOUSEHOLD
[100]	valid_0's rmse: 1.24872
[200]	valid_0's rmse: 1.19546
[300]	valid_0's rmse: 1.18653
[400]	valid_0's rmse: 1.17955
[500]	valid_0's rmse: 1.17242
[600]	valid_0's rmse: 1.16585
[700]	valid_0's rmse: 1.15927
[800]	valid_0's rmse: 1.15351
[900]	valid_0's rmse: 1.14822
[1000]	valid_0's rmse: 1.14256
[1100]	valid_0's rmse: 1.13676
[1200]	valid_0's rmse: 1.1317
[1300]	valid_0's rmse: 1.12676
[1400]	valid_0's rmse: 1.12217
[1500]	valid_0's rmse: 1.1174
[1600]	valid_0's rmse: 1.11287
[1700]	valid_0's rmse: 1.10791
[1800]	valid_0's rmse: 1.10344
[1900]	valid_0's rmse: 1.09932
[2000]	valid_0's rmse: 1.09508
[2100]	valid_0's rmse: 1.09126
[2200]	valid_0's rmse: 1.08712
[2300]	valid_0's rmse: 1.08356
[2400]	valid_0's rmse: 1.08009
[2500]	valid_0's rmse: 1.0763
[2600]	valid_0's rmse: 1.07237
[2700]	valid_0's rmse: 1.06895
[2800]	valid_0's rmse: 1.06519
[2900]	valid_0's rmse: 1.06171
[3000]	valid_0's rmse: 1.0581


,name,imp
0,item_id,130493
14,event_name_1,30501
22,tm_w,24977
21,tm_d,23217
55,rolling_mean_180,21432
56,rolling_std_180,20855
54,rolling_std_60,18731
52,rolling_std_30,17106
50,rolling_std_14,16220
53,rolling_mean_60,16195


Train CA_1 FOODS
[100]	valid_0's rmse: 2.65571
[200]	valid_0's rmse: 2.32775
[300]	valid_0's rmse: 2.28757
[400]	valid_0's rmse: 2.26929
[500]	valid_0's rmse: 2.25433
[600]	valid_0's rmse: 2.24136
[700]	valid_0's rmse: 2.23021
[800]	valid_0's rmse: 2.21866
[900]	valid_0's rmse: 2.20736
[1000]	valid_0's rmse: 2.19603
[1100]	valid_0's rmse: 2.18579
[1200]	valid_0's rmse: 2.17562
[1300]	valid_0's rmse: 2.16564
[1400]	valid_0's rmse: 2.15567
[1500]	valid_0's rmse: 2.14648
[1600]	valid_0's rmse: 2.13706
[1700]	valid_0's rmse: 2.12866
[1800]	valid_0's rmse: 2.11917
[1900]	valid_0's rmse: 2.11024
[2000]	valid_0's rmse: 2.1029
[2100]	valid_0's rmse: 2.09558
[2200]	valid_0's rmse: 2.08753
[2300]	valid_0's rmse: 2.08066
[2400]	valid_0's rmse: 2.07304
[2500]	valid_0's rmse: 2.06535
[2600]	valid_0's rmse: 2.0579
[2700]	valid_0's rmse: 2.04983
[2800]	valid_0's rmse: 2.04214
[2900]	valid_0's rmse: 2.03526
[3000]	valid_0's rmse: 2.02728


...

,name,imp
0,item_id,112130
22,tm_w,26494
21,tm_d,26336
55,rolling_mean_180,22932
56,rolling_std_180,21958
54,rolling_std_60,20848
14,event_name_1,18708
52,rolling_std_30,18126
53,rolling_mean_60,17490
50,rolling_std_14,17353


Train WI_3 HOUSEHOLD
[100]	valid_0's rmse: 1.19982
[200]	valid_0's rmse: 1.13449
[300]	valid_0's rmse: 1.11971
[400]	valid_0's rmse: 1.10904
[500]	valid_0's rmse: 1.09905
[600]	valid_0's rmse: 1.09035
[700]	valid_0's rmse: 1.08156
[800]	valid_0's rmse: 1.07378
[900]	valid_0's rmse: 1.06669
[1000]	valid_0's rmse: 1.05907
[1100]	valid_0's rmse: 1.052
[1200]	valid_0's rmse: 1.04489
[1300]	valid_0's rmse: 1.03881
[1400]	valid_0's rmse: 1.03259
[1500]	valid_0's rmse: 1.02722
[1600]	valid_0's rmse: 1.02153
[1700]	valid_0's rmse: 1.01652
[1800]	valid_0's rmse: 1.0116
[1900]	valid_0's rmse: 1.00629
[2000]	valid_0's rmse: 1.00165
[2100]	valid_0's rmse: 0.996524
[2200]	valid_0's rmse: 0.991852
[2300]	valid_0's rmse: 0.987388
[2400]	valid_0's rmse: 0.983073
[2500]	valid_0's rmse: 0.979054
[2600]	valid_0's rmse: 0.974996
[2700]	valid_0's rmse: 0.970666
[2800]	valid_0's rmse: 0.966961
[2900]	valid_0's rmse: 0.962431
[3000]	valid_0's rmse: 0.958507


,name,imp
0,item_id,125913
14,event_name_1,29910
22,tm_w,26585
21,tm_d,25909
55,rolling_mean_180,22610
56,rolling_std_180,21565
54,rolling_std_60,18681
53,rolling_mean_60,15985
52,rolling_std_30,15772
57,rolling_mean_tmp_1_7,15628


Train WI_3 FOODS
[100]	valid_0's rmse: 2.80402
[200]	valid_0's rmse: 2.40087
[300]	valid_0's rmse: 2.33854
[400]	valid_0's rmse: 2.30532
[500]	valid_0's rmse: 2.28065
[600]	valid_0's rmse: 2.2621
[700]	valid_0's rmse: 2.24147
[800]	valid_0's rmse: 2.2226
[900]	valid_0's rmse: 2.20535
[1000]	valid_0's rmse: 2.18877
[1100]	valid_0's rmse: 2.17048
[1200]	valid_0's rmse: 2.15496
[1300]	valid_0's rmse: 2.13993
[1400]	valid_0's rmse: 2.12731
[1500]	valid_0's rmse: 2.11497
[1600]	valid_0's rmse: 2.10096
[1700]	valid_0's rmse: 2.09051
[1800]	valid_0's rmse: 2.07865
[1900]	valid_0's rmse: 2.06838
[2000]	valid_0's rmse: 2.05675
[2100]	valid_0's rmse: 2.04637
[2200]	valid_0's rmse: 2.0362
[2300]	valid_0's rmse: 2.0233
[2400]	valid_0's rmse: 2.01345
[2500]	valid_0's rmse: 2.00297
[2600]	valid_0's rmse: 1.99514
[2700]	valid_0's rmse: 1.98714
[2800]	valid_0's rmse: 1.978
[2900]	valid_0's rmse: 1.96952
[3000]	valid_0's rmse: 1.96028


,name,imp
0,item_id,145887
14,event_name_1,32829
22,tm_w,23181
21,tm_d,22907
57,rolling_mean_tmp_1_7,17712
55,rolling_mean_180,16533
56,rolling_std_180,15637
58,rolling_mean_tmp_1_14,14904
26,tm_dw,14352
48,rolling_std_7,14116
